<a href="https://colab.research.google.com/github/financieras/big_data/blob/main/leccion_1_1_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lección 1.1.7: El ciclo de vida de los datos en la organización

## 1. Introducción: Los datos como activo vivo

Los datos no son un archivo estático que guardamos y olvidamos. Son un **activo vivo** que fluye continuamente a través de la organización, transformándose en cada etapa, generando valor y eventualmente envejeciendo.

**Lo importante:** Comprender el ciclo de vida completo de los datos es fundamental para diseñar arquitecturas eficientes, garantizar calidad, cumplir regulaciones y extraer el máximo valor. Cada etapa tiene sus propios desafíos, responsabilidades y herramientas.

### La analogía del agua en una ciudad

Piensa en los datos como el agua en una ciudad:

1. **Generación:** Lluvia y manantiales (fuentes de datos)
2. **Recolección:** Captación en embalses (ingesta)
3. **Tratamiento:** Planta potabilizadora (limpieza y transformación)
4. **Distribución:** Red de tuberías (almacenamiento y acceso)
5. **Consumo:** Hogares y empresas (análisis y decisiones)
6. **Reciclaje:** Tratamiento de aguas residuales (archivo)
7. **Eliminación:** Desechos no reutilizables (borrado por compliance)

Al igual que el agua, los datos necesitan **infraestructura, calidad, gobernanza y gestión del ciclo completo**.

### ¿Por qué importa el ciclo de vida?

**Problemas comunes de ignorar el ciclo de vida:**
- 💰 Costes descontrolados (almacenar terabytes de datos que nunca se usan)
- ⚖️ Incumplimiento legal (retener datos más allá de lo permitido por GDPR)
- 🐛 Baja calidad (datos obsoletos o incorrectos en análisis)
- 🔒 Riesgos de seguridad (información sensible no protegida)
- ⏱️ Decisiones lentas (no encontrar los datos cuando se necesitan)

---

## 2. Las 7 etapas del ciclo de vida de los datos

### Visión general

```
1. GENERACIÓN → 2. RECOLECCIÓN → 3. ALMACENAMIENTO → 4. PROCESAMIENTO
                                                              ↓
                                            7. ELIMINACIÓN ← 6. ARCHIVO ← 5. ANÁLISIS/USO
```

| Etapa | Pregunta clave | Responsable principal | Duración |
|-------|----------------|----------------------|----------|
| 1. Generación | ¿De dónde vienen los datos? | Sistemas fuente, usuarios | Continua |
| 2. Recolección | ¿Cómo los capturamos? | Data Engineer | Segundos-horas |
| 3. Almacenamiento | ¿Dónde los guardamos? | Data Engineer | Días-años |
| 4. Procesamiento | ¿Cómo los transformamos? | Data Engineer | Minutos-horas |
| 5. Análisis/Uso | ¿Qué valor extraemos? | Analyst, Scientist | Variable |
| 6. Archivo | ¿Cómo preservamos históricos? | Data Engineer, Architect | Años |
| 7. Eliminación | ¿Cuándo los borramos? | Data Engineer, Legal | Según política |

---

## 3. Etapa 1: Generación de datos

### Definición

La **generación** es el momento en que los datos nacen. Es el origen de todo el ciclo. Los datos pueden generarse de múltiples formas y fuentes.

### Tipos de generación

**1. Datos generados por humanos:**
- Formularios web (registros, compras)
- Emails y mensajes
- Posts en redes sociales
- Documentos y spreadsheets
- Clicks y navegación web

**2. Datos generados por máquinas:**
- Logs de servidores
- Sensores IoT (temperatura, presión, ubicación)
- Transacciones automáticas
- Telemetría de aplicaciones
- Datos de sistemas industriales

**3. Datos derivados:**
- Agregaciones de otros datos
- Features calculadas para ML
- Métricas y KPIs
- Predicciones de modelos

### Ejemplo: Generación de datos en Uber

**Una solicitud de viaje genera:**

```
Momento de solicitud:
├─ Datos del usuario: user_id, ubicación GPS, rating
├─ Datos de la solicitud: timestamp, destino, tipo de vehículo
├─ Datos de contexto: clima, tráfico, eventos cercanos
├─ Datos de precios: estimación inicial, surge pricing

Durante el viaje:
├─ GPS tracking: ubicación cada 5 segundos
├─ Telemetría del conductor: velocidad, aceleración, frenadas
├─ Datos del vehículo: consumo, temperatura

Al finalizar:
├─ Datos de pago: método, monto, propina
├─ Ratings: calificación conductor/pasajero
├─ Datos de viaje: duración real, ruta tomada, distancia
```

**Volumen:** Un viaje promedio genera ~5-10MB de datos. Con 15 millones de viajes diarios = 75-150 TB/día solo de viajes.

### Características de los datos en generación

| Aspecto | Descripción |
|---------|-------------|
| **Velocidad** | Streaming (continua) vs Batch (periódica) |
| **Volumen** | Variable según fuente (bytes a terabytes) |
| **Estructura** | Estructurados, semi-estructurados, no estructurados |
| **Calidad** | Frecuentemente imperfecta (campos vacíos, errores) |
| **Propiedad** | ¿Quién es dueño de estos datos? |

### Consideraciones clave

**Momento de generación = Momento crítico para calidad**

❌ **Error común:** "Ya limpiaremos los datos después"

✅ **Best practice:** Validación en origen (validation at source)

**Ejemplo de validación en origen:**
```python
from pydantic import BaseModel, validator, EmailStr
from datetime import datetime

class UserRegistration(BaseModel):
    email: EmailStr  # Valida formato de email
    age: int
    country_code: str
    timestamp: datetime
    
    @validator('age')
    def age_must_be_reasonable(cls, v):
        if v < 0 or v > 120:
            raise ValueError('Age must be between 0 and 120')
        return v
    
    @validator('country_code')
    def country_code_valid(cls, v):
        valid_codes = ['ES', 'FR', 'DE', 'IT', 'UK', 'US']
        if v not in valid_codes:
            raise ValueError(f'Invalid country code: {v}')
        return v

# Uso
try:
    user = UserRegistration(
        email="user@example.com",
        age=25,
        country_code="ES",
        timestamp=datetime.now()
    )
    # Datos validados, guardar en base de datos
except ValueError as e:
    # Rechazar datos inválidos en el momento de generación
    log_error(f"Invalid data: {e}")
```

**Beneficio:** Prevenir entrada de datos incorrectos ahorra tiempo y dinero en limpieza posterior.

---

## 4. Etapa 2: Recolección (Ingesta)

### Definición

La **recolección o ingesta** es el proceso de capturar datos desde sus fuentes y moverlos hacia los sistemas de la organización para su posterior procesamiento.

### Métodos de ingesta

**1. Batch ingestion (ingesta por lotes):**
- Datos se recolectan en intervalos regulares (horario, diario)
- Más simple y económica
- Latencia alta (horas/días)

**Ejemplo:** Exportación diaria de ventas desde un CRM

```python
# Airflow DAG para ingesta batch diaria
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta

def extract_salesforce_data():
    # Conectar a API de Salesforce
    sales_data = salesforce_api.get_opportunities(
        start_date=datetime.now() - timedelta(days=1),
        end_date=datetime.now()
    )
    # Guardar en S3
    s3.upload(sales_data, f's3://raw-data/salesforce/{datetime.now().date()}.json')

dag = DAG(
    'daily_salesforce_ingestion',
    schedule_interval='0 2 * * *',  # 2 AM diaria
    start_date=datetime(2025, 1, 1)
)

task = PythonOperator(
    task_id='extract_salesforce',
    python_callable=extract_salesforce_data,
    dag=dag
)
```

**2. Real-time/Streaming ingestion (tiempo real):**
- Datos capturados continuamente al generarse
- Mayor complejidad y coste
- Latencia baja (segundos/milisegundos)

**Ejemplo:** Eventos de clicks en una aplicación móvil

```python
# Producer de Kafka para streaming
from kafka import KafkaProducer
import json

producer = KafkaProducer(
    bootstrap_servers=['kafka1:9092', 'kafka2:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

def track_user_event(user_id, event_type, properties):
    event = {
        'user_id': user_id,
        'event_type': event_type,
        'properties': properties,
        'timestamp': datetime.now().isoformat()
    }
    
    # Enviar a Kafka inmediatamente
    producer.send('user_events', value=event)
    producer.flush()

# Cada click se envía instantáneamente
track_user_event(
    user_id='user_123',
    event_type='product_view',
    properties={'product_id': 'prod_456', 'category': 'electronics'}
)
```

**3. Micro-batch ingestion:**
- Híbrido: pequeños lotes frecuentes (cada minuto)
- Balance entre simplicidad y latencia
- Popular con Spark Structured Streaming

### Patrones de ingesta

| Patrón | Cuándo usar | Tecnología típica |
|--------|-------------|------------------|
| **Full load** | Primera carga, tablas pequeñas | Scripts SQL, ETL tools |
| **Incremental** | Solo cambios desde última carga | CDC (Change Data Capture) |
| **Delta/Upsert** | Actualizaciones y nuevos registros | Merge statements, Delta Lake |
| **Event streaming** | Eventos en tiempo real | Kafka, Kinesis, Pub/Sub |
| **API polling** | SaaS tools (Salesforce, Stripe) | Fivetran, Airbyte, custom scripts |
| **File drop** | Vendors externos | S3 event triggers, file watchers |

### Ejemplo real: Ingesta en Spotify

**Desafío:** Ingestar datos de reproducción de 500M+ usuarios activos.

**Arquitectura de ingesta:**

```
Aplicación móvil/web
    ↓ (eventos de reproducción)
Kafka (100M eventos/hora)
    ↓
Spark Streaming (procesamiento inicial)
    ↓
S3 (data lake - formato Parquet)
    ↓
Snowflake (data warehouse para análisis)
```

**Datos generados:**
- **Event:** Play, pause, skip, like, add to playlist
- **Metadata:** Song ID, artist, album, timestamp, device, location
- **Volumen:** ~10 TB/día

**Decisiones técnicas:**
- Streaming para eventos de reproducción (necesitan actualizarse constantemente)
- Batch para metadata de canciones (cambia raramente)
- Particionamiento por fecha y región para optimizar queries

### Desafíos comunes en recolección

**1. Manejo de fallos:**

❌ **Mal enfoque:** Si falla, se pierden datos

✅ **Buen enfoque:** Retry logic + dead letter queues

```python
from tenacity import retry, stop_after_attempt, wait_exponential

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=4, max=10)
)
def ingest_from_api():
    try:
        data = api.fetch_data()
        store_in_database(data)
    except Exception as e:
        # Si falla después de 3 intentos, enviar a dead letter queue
        dead_letter_queue.send(data)
        raise
```

**2. Idempotencia:**
- Ejecutar la ingesta múltiples veces debe dar el mismo resultado
- Crítico para reprocessing

**3. Schema evolution:**
- Las fuentes de datos cambian con el tiempo
- Necesitas manejar campos nuevos, eliminados o modificados

**4. Rate limits:**
- APIs externas tienen límites de requests
- Implementar throttling y backoff

---

## 5. Etapa 3: Almacenamiento

### Definición

El **almacenamiento** es donde los datos residen después de ser recolectados, esperando ser procesados o consultados.

### Tipos de almacenamiento por propósito

**1. Transactional databases (OLTP):**
- **Propósito:** Operaciones diarias del negocio
- **Características:** ACID, low latency, muchas escrituras pequeñas
- **Tecnologías:** PostgreSQL, MySQL, MongoDB
- **Ejemplo:** Base de datos de pedidos de e-commerce

**2. Data warehouses (OLAP):**
- **Propósito:** Análisis y reporting
- **Características:** Optimizado para lecturas, consultas complejas, datos históricos
- **Tecnologías:** Snowflake, BigQuery, Redshift
- **Ejemplo:** Warehouse de ventas históricas para análisis de tendencias

**3. Data lakes:**
- **Propósito:** Almacenamiento masivo de datos raw
- **Características:** Schema-on-read, cualquier formato, bajo coste
- **Tecnologías:** S3, Azure Data Lake Storage, Google Cloud Storage
- **Ejemplo:** Logs de aplicación, imágenes, datos no estructurados

**4. Data lakehouses:**
- **Propósito:** Unificar lake + warehouse
- **Características:** Flexibilidad del lake + performance del warehouse
- **Tecnologías:** Databricks (Delta Lake), Snowflake, Iceberg
- **Ejemplo:** Plataforma unificada para analytics y ML

### Arquitectura medallion (Lakehouse)

Patrón moderno para organizar datos en capas:

```
┌─────────────────────────────────────────┐
│  BRONZE (Raw data)                      │
│  - Datos tal cual llegan                │
│  - Sin transformaciones                 │
│  - Schema mínimo o inexistente          │
│  - Retención: 30-90 días                │
└─────────────────────────────────────────┘
            ↓ (limpieza básica)
┌─────────────────────────────────────────┐
│  SILVER (Cleaned data)                  │
│  - Datos validados y limpios            │
│  - Schema definido                      │
│  - Duplicados eliminados                │
│  - Retención: 1-2 años                  │
└─────────────────────────────────────────┘
            ↓ (agregaciones)
┌─────────────────────────────────────────┐
│  GOLD (Business-level aggregates)       │
│  - Datos listos para negocio            │
│  - KPIs, métricas, features para ML     │
│  - Altamente optimizados                │
│  - Retención: Indefinida                │
└─────────────────────────────────────────┘
```

**Ejemplo en código (dbt):**

```sql
-- bronze/raw_orders.sql
-- Simplemente copia de la fuente
SELECT * FROM source_db.orders

-- silver/clean_orders.sql
-- Limpieza y validación
SELECT
    order_id,
    customer_id,
    order_date,
    CAST(amount AS DECIMAL(10,2)) as amount,
    status
FROM {{ ref('raw_orders') }}
WHERE
    amount > 0
    AND customer_id IS NOT NULL
    AND order_date >= '2020-01-01'

-- gold/daily_revenue.sql
-- Agregación para negocio
SELECT
    DATE(order_date) as date,
    COUNT(DISTINCT order_id) as orders,
    COUNT(DISTINCT customer_id) as customers,
    SUM(amount) as revenue,
    AVG(amount) as avg_order_value
FROM {{ ref('clean_orders') }}
WHERE status = 'completed'
GROUP BY DATE(order_date)
```

### Hot, warm, cold data

Estrategia de almacenamiento según frecuencia de acceso:

| Tier | Descripción | Tiempo de acceso | Coste | Ejemplo |
|------|-------------|------------------|-------|---------|
| **Hot** | Acceso frecuente | Milisegundos | Alto | Datos últimos 7 días |
| **Warm** | Acceso ocasional | Segundos | Medio | Datos 7-90 días |
| **Cold** | Acceso raro | Minutos-horas | Bajo | Datos >90 días |
| **Archive** | Casi nunca | Horas | Muy bajo | Datos >1 año, compliance |

**Ejemplo de políticas en AWS S3:**

```python
# Lifecycle policy para S3
lifecycle_policy = {
    'Rules': [
        {
            'Id': 'Move to Glacier',
            'Status': 'Enabled',
            'Transitions': [
                {
                    'Days': 90,
                    'StorageClass': 'GLACIER'  # Warm → Cold
                },
                {
                    'Days': 365,
                    'StorageClass': 'DEEP_ARCHIVE'  # Cold → Archive
                }
            ]
        }
    ]
}

s3_client.put_bucket_lifecycle_configuration(
    Bucket='my-data-lake',
    LifecycleConfiguration=lifecycle_policy
)
```

**Ahorro de costes:** Mover datos antiguos a tiers fríos puede reducir costes de almacenamiento en 70-90%.

### Ejemplo real: Almacenamiento en Netflix

**Arquitectura de almacenamiento:**

```
Transactional DBs (MySQL, Cassandra)
    ↓ (CDC - Change Data Capture)
S3 Data Lake (Bronze - Parquet)
    ↓ (Spark jobs)
S3 Data Lake (Silver/Gold - Iceberg tables)
    ↓ (query engine)
Redshift (agregados para dashboards)
    ↓
Tableau/Looker (visualización)
```

**Volúmenes:**
- **S3:** 100+ petabytes de datos históricos
- **Cassandra:** Decenas de TB de datos operacionales (perfiles de usuario, viewing history reciente)
- **Redshift:** Varios TB de datos agregados para analytics

**Decisiones de almacenamiento:**
- **Bronze (raw logs):** Retención 30 días en hot, después a Glacier
- **Silver (processed):** Retención 2 años en S3 standard
- **Gold (aggregates):** Indefinida, particionado por fecha para performance

---

## 6. Etapa 4: Procesamiento y transformación

### Definición

El **procesamiento** es donde los datos raw se transforman en datos útiles para análisis. Es la "cocina" donde se preparan los datos.

### Tipos de procesamiento

**1. Batch processing:**
- Procesar grandes volúmenes en bloques
- Schedule regular (diario, horario)
- Tecnologías: Spark, dbt, SQL en warehouses

**2. Stream processing:**
- Procesar datos evento por evento en tiempo real
- Latencia baja
- Tecnologías: Flink, Kafka Streams, Spark Streaming

**3. Micro-batch:**
- Híbrido: pequeños batches frecuentes
- Balance entre throughput y latencia

### Operaciones comunes de transformación

**1. Limpieza (Cleaning):**
```python
import pandas as pd

# Eliminar duplicados
df = df.drop_duplicates(subset=['user_id', 'timestamp'])

# Manejar valores faltantes
df['age'].fillna(df['age'].median(), inplace=True)

# Corregir tipos de datos
df['order_date'] = pd.to_datetime(df['order_date'])

# Eliminar outliers
Q1 = df['revenue'].quantile(0.25)
Q3 = df['revenue'].quantile(0.75)
IQR = Q3 - Q1
df = df[(df['revenue'] >= Q1 - 1.5*IQR) & (df['revenue'] <= Q3 + 1.5*IQR)]
```

**2. Enriquecimiento (Enrichment):**
- Añadir información de otras fuentes
- Geocoding (agregar país a partir de IP)
- Lookups (agregar nombre de producto a partir de ID)

```sql
-- Enriquecer transacciones con datos de cliente
SELECT
    t.transaction_id,
    t.amount,
    c.customer_name,
    c.customer_segment,
    c.lifetime_value
FROM transactions t
LEFT JOIN customers c ON t.customer_id = c.customer_id
```

**3. Agregación:**
- Resumir datos a nivel superior
- De transacciones → métricas diarias
- De eventos → sesiones de usuario

```sql
-- Agregación de transacciones a nivel diario
SELECT
    DATE(transaction_date) as date,
    customer_id,
    COUNT(*) as num_transactions,
    SUM(amount) as total_spent,
    AVG(amount) as avg_transaction_value
FROM transactions
GROUP BY DATE(transaction_date), customer_id
```

**4. Feature engineering (para ML):**
- Crear variables predictivas

```python
# Features de tiempo
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Features de comportamiento
df['days_since_last_purchase'] = (
    df.groupby('customer_id')['purchase_date']
    .diff()
    .dt.days
)

# Ratios
df['revenue_per_item'] = df['total_revenue'] / df['num_items']

# Encoding categórico
df = pd.get_dummies(df, columns=['category'], prefix='cat')
```

### ETL vs ELT

**ETL (Extract-Transform-Load):**
```
Source → Transform (en servidor intermedio) → Load (warehouse)
```
- Tradicional
- Transformación antes de cargar
- Más lento pero datos llegan limpios

**ELT (Extract-Load-Transform):**
```
Source → Load (warehouse raw) → Transform (dentro del warehouse)
```
- Moderno
- Aprovechar poder de procesamiento del warehouse
- Más rápido, más flexible
- **dbt es el estándar para ELT**

**Ejemplo dbt (ELT moderno):**

```sql
-- models/staging/stg_orders.sql
with source as (
    select * from {{ source('ecommerce', 'raw_orders') }}
),

cleaned as (
    select
        order_id,
        customer_id,
        order_date::date as order_date,
        status,
        case
            when amount < 0 then 0
            else amount
        end as amount
    from source
    where order_date >= '2020-01-01'
)

select * from cleaned
```

### Data quality checks

**Implementar validaciones en el procesamiento:**

```python
# Great Expectations para validación de datos
import great_expectations as ge

df = ge.read_csv('orders.csv')

# Expectations (reglas de calidad)
df.expect_column_values_to_not_be_null('order_id')
df.expect_column_values_to_be_unique('order_id')
df.expect_column_values_to_be_between('amount', min_value=0, max_value=100000)
df.expect_column_values_to_be_in_set('status', ['pending', 'completed', 'cancelled'])

# Validar
results = df.validate()

if not results.success:
    # Alertar si falla la calidad
    send_alert(f"Data quality check failed: {results}")
    raise Exception("Data quality issues detected")
```

### Ejemplo real: Pipeline de procesamiento en Airbnb

**Desafío:** Procesar datos de búsquedas, bookings y reviews para generar métricas de negocio.

**Pipeline:**

```
1. INGESTA (Kafka)
   - Eventos de búsqueda (1M/hora)
   - Eventos de booking (50K/día)
   - Reviews (10K/día)
         ↓
2. PROCESAMIENTO STREAM (Flink)
   - Detección de fraude en tiempo real
   - Actualización de availability
   - Alertas operacionales
         ↓
3. BATCH PROCESSING (Spark + dbt)
   - Cálculo de métricas de hosts (nightly)
   - Agregación de ocupación por ciudad
   - Features para modelo de pricing dinámico
         ↓
4. GOLD TABLES
   - daily_bookings_by_city
   - host_performance_metrics
   - search_to_booking_funnel
```

**Frecuencia:**
- Stream processing: Continuo (latencia `<5` segundos)
- Batch processing: Diario a las 2 AM
- Feature engineering para ML: Semanal

---

## 7. Etapa 5: Análisis y uso

### Definición

El **análisis** es donde finalmente extraemos valor de los datos, respondiendo preguntas de negocio, creando visualizaciones y entrenando modelos.

### Tipos de análisis

**1. Análisis descriptivo (¿Qué pasó?):**
- Dashboards de KPIs
- Reportes de ventas, usuarios, engagement
- **Herramientas:** Tableau, Power BI, Looker

**2. Análisis diagnóstico (¿Por qué pasó?):**
- Root cause analysis
- Segmentación de clientes
- Análisis de cohortes
- **Herramientas:** SQL, Python (Pandas)

**3. Análisis predictivo (¿Qué pasará?):**
- Forecasting de demanda
- Predicción de churn
- Credit scoring
- **Herramientas:** Scikit-learn, XGBoost, Prophet

**4. Análisis prescriptivo (¿Qué deberíamos hacer?):**
- Optimización de rutas
- Pricing dinámico
- Recomendaciones personalizadas
- **Herramientas:** OR-Tools, ML models, A/B testing

### Democratización de datos

**Self-service analytics:**

Permitir que usuarios no técnicos accedan y analicen datos sin depender de Data Engineers.

**Componentes clave:**

```
Data Warehouse (Snowflake, BigQuery)
        ↓
Semantic Layer (Looker, dbt Metrics)
        ↓
BI Tools (Tableau, Metabase)
        ↓
Business Users
```

**Beneficios:**
- ✅ Analistas responden sus propias preguntas
- ✅ Engineers se liberan para infraestructura
- ✅ Decisiones más rápidas

**Riesgos:**
- ❌ Interpretación incorrecta de métricas
- ❌ Queries ineficientes que ralentizan el warehouse
- ❌ Proliferación de dashboards redundantes

**Solución:** Semantic layer con definiciones centralizadas

```yaml
# dbt metric definition
metrics:
  - name: monthly_revenue
    label: Monthly Revenue
    model: ref('orders')
    type: sum
    sql: amount
    timestamp: order_date
    time_grains: [day, week, month]
    dimensions:
      - country
      - product_category
    filters:
      - field: status
        operator: '='
        value: "'completed'"
```

**Resultado:** Todos usan la misma definición de "Monthly Revenue", reduciendo discrepancias.

### Ejemplo de uso: Data-driven decision en Amazon

**Pregunta de negocio:** ¿Deberíamos ofrecer envío gratuito en pedidos >30€?

**Análisis realizado:**

**1. Análisis descriptivo:**
- 45% de pedidos están entre 25-30€
- AOV (Average Order Value) actual: 28€

**2. Análisis diagnóstico:**
- Clientes con pedidos 25-30€ tienen 2.3x más probabilidad de abandonar carrito
- Coste de envío es la razón #1 de abandono

**3. Análisis predictivo:**
- Modelo predice: envío gratuito >30€ aumentaría AOV a 32€
- 15% de clientes subirían su pedido para alcanzar umbral

**4. A/B Test:**
- Grupo control: sin cambios
- Grupo tratamiento: envío gratis >30€
- Duración: 4 semanas

**Resultados:**
- AOV subió 14% (28€ → 32€)
- Tasa de conversión +8%
- Revenue neto +12% (después de restar coste de envíos gratuitos)

**Decisión:** Implementar envío gratuito >30€ en toda la plataforma.

---

## 8. Etapa 6: Archivo (Archival)

### Definición

El **archivo** es el almacenamiento a largo plazo de datos históricos que ya no se usan activamente pero deben preservarse por razones legales, de compliance o análisis histórico eventual.

### ¿Cuándo archivar?

**Señales para archivar:**
- Datos con `>1` año de antigüedad
- Acceso `<1` vez al mes
- Requeridos por ley (retención obligatoria)
- Necesarios para auditorías
- Datos de entrenamiento de modelos históricos

### Estrategias de archivo

**1. Cold storage en cloud:**

```python
# AWS S3 Glacier Deep Archive
s3_client.copy_object(
    CopySource={'Bucket': 'hot-data', 'Key': 'orders_2020.parquet'},
    Bucket='archive',
    Key='orders_2020.parquet',
    StorageClass='DEEP_ARCHIVE'  # Muy bajo coste, retrieval en horas
)
```

**Comparativa de costes AWS S3:**
- Standard: $0.023/GB/mes
- Glacier: $0.004/GB/mes (↓83%)
- Deep Archive: $0.00099/GB/mes (↓96%)

**2. Compresión:**

```python
# Comprimir antes de archivar
import pyarrow.parquet as pq
import pyarrow as pa

# Leer datos
df = pd.read_csv('large_file.csv')

# Escribir con alta compresión
pq.write_table(
    pa.Table.from_pandas(df),
    'large_file.parquet',
    compression='zstd',  # Mejor compresión que gzip
    compression_level=19  # Máxima compresión
)
# Reducción típica: 70-90% del tamaño original
```

**3. Particionamiento inteligente:**

```
archive/
├── year=2020/
│   ├── month=01/
│   │   └── data.parquet.gz
│   ├── month=02/
│   └── ...
├── year=2021/
└── year=2022/
```

**Beneficio:** Recuperar solo lo necesario sin descargar todo el archivo.

### Data retention policies (políticas de retención)

**Definir cuánto tiempo mantener cada tipo de dato:**

| Tipo de dato | Retención hot | Retención archive | Motivo |
|--------------|---------------|-------------------|--------|
| Transacciones financieras | 3 meses | 7 años | Ley (obligatorio) |
| Logs de aplicación | 30 días | 1 año | Debugging |
| Datos de usuarios | Mientras activo | N/A | GDPR (derecho al olvido) |
| Datos de training ML | 1 año | 5 años | Reproducibilidad |
| Emails | 90 días | 3 años | Compliance |

**Ejemplo de política automatizada:**

```yaml
# Data retention policy
retention_policies:
  - dataset: financial_transactions
    hot_retention_days: 90
    archive_retention_years: 7
    deletion_allowed: false  # Nunca borrar
    
  - dataset: application_logs
    hot_retention_days: 30
    archive_retention_years: 1
    deletion_allowed: true
    
  - dataset: user_clickstream
    hot_retention_days: 60
    archive_retention_years: 2
    deletion_allowed: true
```

### Ejemplo real: Archivo en una entidad bancaria

**Requisitos:**
- Regulación obliga retener transacciones 10 años
- Auditorías requieren acceso a datos históricos
- Datos antiguos casi nunca se consultan

**Solución implementada:**

```
Año 0-1: PostgreSQL (hot - consultas diarias)
    ↓ (automated migration)
Año 1-3: Redshift (warm - consultas ocasionales)
    ↓
Año 3-10: S3 Glacier (cold - solo auditorías)
    ↓
Año 10+: Eliminación automática (cumplido el requisito legal)
```

**Coste:**
- **Antes de archivo:** $50K/mes en almacenamiento hot
- **Después:** $5K/mes (↓90% en costes)

**Tiempo de recuperación:**
- Año 1-3: `<1` minuto (Redshift)
- Año 3-10: 12-48 horas (Glacier restore)
- Aceptable porque accesos son raros

---

## 9. Etapa 7: Eliminación (Deletion)

### Definición

La **eliminación** es el borrado permanente de datos que ya no son necesarios ni legalmente requeridos. Es la etapa final del ciclo de vida.

### ¿Por qué eliminar datos?

**1. Compliance legal:**
- GDPR: Derecho al olvido (Right to be forgotten)
- CCPA (California): Derecho a eliminación
- Evitar retención excesiva de datos personales

**2. Reducción de riesgos:**
- Menos datos = menor superficie de ataque
- Reducir riesgo de data breaches
- Datos que no tienes no pueden filtrarse

**3. Reducción de costes:**
- Almacenar datos innecesarios cuesta dinero
- Menor complejidad operacional

### Eliminación lógica vs física

**Eliminación lógica (soft delete):**
```sql
-- Marcar como eliminado, pero mantener el registro
UPDATE users
SET deleted_at = NOW(),
    status = 'deleted'
WHERE user_id = '12345';

-- Las queries ignoran registros eliminados
SELECT * FROM users WHERE deleted_at IS NULL;
```

**Ventajas:**
- ✅ Recuperable si fue error
- ✅ Mantiene integridad referencial
- ✅ Auditable

**Desventajas:**
- ❌ Datos siguen ocupando espacio
- ❌ No cumple GDPR estricto

**Eliminación física (hard delete):**
```sql
-- Borrado permanente
DELETE FROM users WHERE user_id = '12345';
-- Eliminar todas las referencias en cascada
DELETE FROM orders WHERE user_id = '12345';
DELETE FROM reviews WHERE user_id = '12345';
```

**Ventajas:**
- ✅ Datos verdaderamente eliminados
- ✅ Cumple GDPR
- ✅ Libera almacenamiento

**Desventajas:**
- ❌ Irreversible
- ❌ Puede romper integridad si no se hace bien
- ❌ Complejo en sistemas distribuidos

### Desafíos de eliminación

**1. Datos distribuidos:**

Datos de un usuario pueden estar en 20+ tablas/sistemas:
```
User data ubicaciones:
├─ PostgreSQL (users table)
├─ MongoDB (preferences)
├─ Elasticsearch (search index)
├─ S3 (uploaded files)
├─ Redshift (analytics warehouse)
├─ Redis (cache)
├─ Kafka (event logs)
└─ Backups (múltiples ubicaciones)
```

**Solución:** Data mapping completo + proceso orquestado

```python
# Orquestación de eliminación GDPR
class GDPRDeletionService:
    def delete_user_data(self, user_id):
        deleted_locations = []
        
        # 1. Eliminar de sistemas operacionales
        postgres.delete(f"DELETE FROM users WHERE user_id = '{user_id}'")
        deleted_locations.append('postgres')
        
        # 2. Eliminar de analytics
        redshift.delete(f"DELETE FROM dim_users WHERE user_id = '{user_id}'")
        deleted_locations.append('redshift')
        
        # 3. Eliminar archivos en S3
        s3.delete_objects_by_prefix(f"user_data/{user_id}/")
        deleted_locations.append('s3')
        
        # 4. Limpiar caché
        redis.delete(f"user:{user_id}")
        deleted_locations.append('redis')
        
        # 5. Eliminar de índices de búsqueda
        elasticsearch.delete_by_query(f"user_id:{user_id}")
        deleted_locations.append('elasticsearch')
        
        # 6. Registrar en audit log
        audit_log.record({
            'action': 'gdpr_deletion',
            'user_id': user_id,
            'timestamp': datetime.now(),
            'locations': deleted_locations
        })
        
        return {'success': True, 'locations_deleted': deleted_locations}
```

**2. Backups:**

Desafío: Datos eliminados pueden seguir en backups antiguos.

**Solución:**
- Encriptar backups con claves que se pueden destruir
- Implementar "crypto-shredding" (destruir clave = datos ilegibles)
- Retención limitada de backups (30-90 días)

**3. Anonymización como alternativa:**

En lugar de eliminar, anonimizar para analytics:

```python
def anonymize_user(user_id):
    # Reemplazar PII con valores sintéticos pero estadísticamente similares
    user = get_user(user_id)
    
    anonymized = {
        'user_id': hash(user_id),  # Hash irreversible
        'age': round_to_range(user.age),  # 25 → "20-30"
        'country': user.country,  # Mantener (no identificable)
        'email': None,  # Eliminar completamente
        'name': None,
        'purchase_history': [
            {'amount': p.amount, 'category': p.category}
            # Mantener montos y categorías, eliminar detalles específicos
            for p in user.purchases
        ]
    }
    
    return anonymized
```

**Beneficio:** Mantener utilidad analítica sin identificar individuos.

### Políticas de eliminación automatizada

```yaml
# Automated deletion policy
deletion_policies:
  - trigger: user_account_deletion_request
    retention_period: 30_days  # Grace period
    cascade_delete:
      - users
      - orders
      - reviews
      - uploads
    notify:
      - user_email
      - compliance_team
    
  - trigger: data_retention_exceeded
    datasets:
      - application_logs: 90_days
      - marketing_analytics: 2_years
    schedule: daily_at_2am
```

### Ejemplo real: GDPR deletion en Facebook/Meta

**Complejidad:**
- Datos de un usuario en 100+ sistemas internos
- Millones de interacciones con otros usuarios (likes, comments, tags)
- Backups distribuidos globalmente

**Solución implementada:**
1. **User request:** Usuario solicita borrado desde configuración
2. **Grace period:** 30 días para arrepentirse (cuenta suspendida pero datos intactos)
3. **Cascading deletion:** Proceso de 90 días
   - Día 1-30: Eliminar de sistemas de cara al usuario
   - Día 31-60: Eliminar de sistemas de analytics
   - Día 61-90: Eliminar de backups y logs
4. **Anonymization:** Algunos datos agregados se anonimizan en lugar de eliminar
5. **Audit trail:** Todo el proceso registrado para auditorías

**Desafío continuo:** Backups pre-GDPR aún contienen datos antiguos hasta que expiran.

---

## 10. Gobernanza del ciclo de vida

### Data governance framework

La **gobernanza** asegura que cada etapa del ciclo de vida cumpla estándares de calidad, seguridad y compliance.

### Componentes de gobernanza

**1. Data lineage (linaje de datos):**

Rastrear el origen y transformaciones de cada dato.

```
Source: Salesforce API
    ↓ (Ingestion: Fivetran)
Bronze: s3://raw/salesforce/opportunities/
    ↓ (Transformation: dbt)
Silver: s3://cleaned/opportunities/
    ↓ (Aggregation: Spark)
Gold: redshift.gold.daily_sales
    ↓ (Visualization: Tableau)
Dashboard: "Sales Performance"
```

**Herramientas:** OpenLineage, DataHub, Apache Atlas

**Beneficio:**
- Debugging: "¿Por qué este número cambió?"
- Impact analysis: "Si modifico esta tabla, ¿qué se rompe?"
- Compliance: "Demostrar origen de datos en auditoría"

**2. Data quality monitoring:**

Monitorizar calidad continuamente:

```python
# Great Expectations checkpoint
checkpoint_config = {
    'name': 'daily_orders_quality_check',
    'expectations': [
        {
            'expectation_type': 'expect_column_values_to_not_be_null',
            'column': 'order_id'
        },
        {
            'expectation_type': 'expect_column_values_to_be_between',
            'column': 'amount',
            'min_value': 0,
            'max_value': 100000
        },
        {
            'expectation_type': 'expect_table_row_count_to_be_between',
            'min_value': 10000,
            'max_value': 100000
        }
    ],
    'actions': [
        {
            'name': 'send_slack_alert_on_failure',
            'webhook': 'https://hooks.slack.com/...'
        }
    ]
}
```

**3. Data catalog:**

Inventario centralizado de todos los datos:

```yaml
# Data catalog entry
dataset: customers
description: "Customer master data"
owner: data-platform-team
pii_data: true
retention_policy: 3_years
update_frequency: daily
schema:
  - name: customer_id
    type: string
    description: "Unique customer identifier"
    pii: false
  - name: email
    type: string
    description: "Customer email"
    pii: true
  - name: lifetime_value
    type: decimal
    description: "Total revenue from customer"
    pii: false
tags:
  - customer-data
  - pii
  - tier-1
```

**Herramientas:** Alation, Collibra, DataHub

**4. Access control:**

Quién puede acceder a qué datos:

```sql
-- Row-level security en Snowflake
CREATE ROW ACCESS POLICY customer_data_policy AS (customer_country STRING)
RETURNS BOOLEAN ->
  CASE
    WHEN CURRENT_ROLE() IN ('ADMIN', 'COMPLIANCE') THEN TRUE
    WHEN CURRENT_ROLE() = 'EU_ANALYST' AND customer_country IN ('ES', 'FR', 'DE') THEN TRUE
    WHEN CURRENT_ROLE() = 'US_ANALYST' AND customer_country = 'US' THEN TRUE
    ELSE FALSE
  END;

ALTER TABLE customers
ADD ROW ACCESS POLICY customer_data_policy ON (country);
```

**5. Audit logging:**

Registrar todos los accesos a datos sensibles:

```python
# Audit log entry
{
    'timestamp': '2025-10-06T14:23:15Z',
    'user': 'analyst@company.com',
    'action': 'SELECT',
    'table': 'customers',
    'columns': ['email', 'name', 'lifetime_value'],
    'rows_accessed': 15023,
    'query': 'SELECT email, name, lifetime_value FROM customers WHERE country = ES',
    'ip_address': '192.168.1.100',
    'approved': true
}
```

### Roles y responsabilidades

| Rol | Responsabilidad en gobernanza |
|-----|------------------------------|
| **Data Steward** | Define reglas de calidad y acceso |
| **Data Engineer** | Implementa controles técnicos |
| **Data Analyst** | Reporta problemas de calidad |
| **Legal/Compliance** | Define políticas de retención y privacidad |
| **Security** | Implementa controles de acceso |
| **Data Architect** | Diseña arquitectura de gobernanza |

---

## 11. Métricas del ciclo de vida

### KPIs para monitorizar

**1. Data freshness (frescura):**
- ¿Cuán actualizados están los datos?
- Métrica: `current_time - last_update_time`
- Target: <24h para reportes, <1h para operacional

**2. Data quality score:**
- % de registros que pasan validaciones
- Target: >99%

**3. Storage costs:**
- Coste por TB almacenado
- Tendencia a lo largo del tiempo
- Target: Reducción 10% anual por optimizaciones

**4. Time to insights:**
- Desde generación hasta disponible para análisis
- Target: <24h para batch, <5min para streaming

**5. Data coverage:**
- % de datos que tienen metadata y lineage
- Target: >90%

**6. Compliance rate:**
- % de datasets cumpliendo políticas de retención
- Target: 100%

### Dashboard de monitorización

```
┌─────────────────────────────────────────────────────────┐
│  Data Platform Health Dashboard                         │
├─────────────────────────────────────────────────────────┤
│  Data Freshness:           ✅ 98% < 24h                 │
│  Quality Score:            ⚠️  96.5% (target: 99%)       │
│  Storage Growth:           📈 +15% MoM                  │
│  Failed Pipelines:         ❌ 3 in last 24h             │
│  GDPR Deletion Requests:   ⏱️  12 pending               │
│  Data Catalog Coverage:    ✅ 94%                       │
└─────────────────────────────────────────────────────────┘
```

---

## 12. Conceptos clave

- **Ciclo de vida:** Datos pasan por 7 etapas desde generación hasta eliminación
- **Generación:** Momento crítico para calidad (validar en origen)
- **Ingesta:** Batch vs streaming según latencia requerida
- **Almacenamiento:** Medallion architecture (Bronze → Silver → Gold)
- **Hot/Warm/Cold:** Estratificar datos según frecuencia de acceso
- **Procesamiento:** ETL tradicional vs ELT moderno (dbt)
- **Data quality:** Validaciones automatizadas en cada etapa
- **Archivo:** Reducir costes moviendo datos antiguos a cold storage
- **Eliminación:** GDPR exige capacidad de borrado completo
- **Data lineage:** Rastrear origen y transformaciones para debugging y compliance
- **Gobernanza:** Políticas, controles y monitorización a lo largo del ciclo
- **Idempotencia:** Procesos repetibles con mismo resultado
- **Retención:** Políticas claras de cuánto tiempo mantener cada tipo de dato

---

## Resumen

El ciclo de vida de los datos en la organización es un flujo continuo que abarca siete etapas interdependientes: generación, recolección, almacenamiento, procesamiento, análisis/uso, archivo y eliminación. Comprender este ciclo completo es fundamental para diseñar arquitecturas de datos eficientes, garantizar calidad, cumplir regulaciones y maximizar el valor extraído.

Los datos nacen en la **generación**, donde la validación temprana previene problemas posteriores. Son capturados mediante **ingesta** (batch o streaming según latencia requerida) y **almacenados** en arquitecturas modernas como data lakehouses con estrategia medallion (Bronze-Silver-Gold). El **procesamiento** transforma datos raw en información útil, siguiendo cada vez más el paradigma ELT con herramientas como dbt. El **análisis** extrae valor mediante descriptive, predictive y prescriptive analytics. Los datos históricos se mueven a **archivo** (cold storage) reduciendo costes 70-90%, y finalmente se **eliminan** cuando ya no son necesarios o requeridos legalmente.

La **gobernanza** atraviesa todas las etapas: data lineage rastrea transformaciones, quality monitoring detecta anomalías, data catalogs documentan assets, y access controls protegen información sensible. Políticas de retención claras, implementadas técnicamente con lifecycle policies automatizadas, equilibran valor de negocio con compliance (especialmente GDPR).

El éxito no está en cada etapa aislada, sino en la orquestación fluida del ciclo completo: datos que fluyen eficientemente desde su origen hasta generar insights accionables, archivándose cuando pierden relevancia y eliminándose cuando es necesario, todo bajo un framework robusto de gobernanza que asegura calidad, seguridad y cumplimiento legal.

---

## Referencias

### Vídeos
- [Data Lifecycle Management Explained](https://www.youtube.com/results?search_query=data+lifecycle+management)
- [What is Data Lineage?](https://www.youtube.com/results?search_query=data+lineage+explained)
- [ETL vs ELT: Which is Better?](https://www.youtube.com/results?search_query=etl+vs+elt)
- [GDPR and Data Deletion Requirements](https://www.youtube.com/results?search_query=gdpr+data+deletion)
- [Building a Data Lakehouse](https://www.youtube.com/results?search_query=data+lakehouse+architecture)

### Lecturas
- [The Medallion Architecture - Databricks](https://www.databricks.com/glossary/medallion-architecture)
- [Data Lifecycle Management Best Practices - AWS](https://aws.amazon.com/blogs/storage/best-practices-for-data-lifecycle-management-on-amazon-s3/)
- [Modern Data Stack Explained - dbt](https://www.getdbt.com/blog/future-of-the-modern-data-stack/)
- [GDPR Right to Erasure - Official Guide](https://gdpr-info.eu/art-17-gdpr/)
- [Data Quality in the Modern Data Stack](https://www.montecarlodata.com/blog-what-is-data-quality/)

### Herramientas
- [Great Expectations - Data Quality](https://greatexpectations.io/)
- [dbt - Data Transformation](https://www.getdbt.com/)
- [Apache Airflow - Orchestration](https://airflow.apache.org/)
- [DataHub - Data Catalog & Lineage](https://datahubproject.io/)
- [Delta Lake - Lakehouse Storage](https://delta.io/)
- [OpenLineage - Lineage Standard](https://openlineage.io/)

### Guías prácticas
- [AWS S3 Lifecycle Configuration](https://docs.aws.amazon.com/AmazonS3/latest/userguide/object-lifecycle-mgmt.html)
- [Implementing GDPR Data Deletion](https://www.atlassian.com/blog/confluence/gdpr-data-deletion-guide)
- [Building Data Pipelines with Airflow](https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html)
- [Data Governance Framework - DAMA](https://www.dama.org/cpages/body-of-knowledge)